<a href="https://colab.research.google.com/github/Lahari033/SRU_Batch-17-18/blob/main/week10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# -----------------------------
# Sample data (given in question)
# -----------------------------
data = pd.DataFrame({
    'category': ['Electronics', 'Clothing']*50,
    'region': ['North', 'South', 'East', 'West']*25,
    'sales_volume': [100, 200, 150, 300]*25,
    'revenue': [5000, 8000, 6000, 12000]*25,
    'customer_rating': [4.5, 3.8, 4.2, 4.0]*25,
    'month': ['Jan', 'Feb', 'Mar', 'Apr']*25
})

# -----------------------------
# Create subplot layout
# -----------------------------
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Sales Volume vs Revenue", "Seasonal Trends by Category"),
    column_widths=[0.6, 0.4]
)

# -----------------------------
# Scatter Plot: Sales Volume vs Revenue
# -----------------------------
months = sorted(data['month'].unique())
scatter_traces = []

for month in months:
    df_month = data[data['month'] == month]
    scatter = go.Scatter(
        x=df_month['sales_volume'],
        y=df_month['revenue'],
        mode='markers',
        name=month,
        marker=dict(
            size=10,
            color=df_month['region'].astype('category').cat.codes,
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Region Code")
        ),
        text=[f"Region: {r}<br>Rating: {rt}" for r, rt in zip(df_month['region'], df_month['customer_rating'])],
        hovertemplate="Sales Volume: %{x}<br>Revenue: %{y}<br>%{text}<extra></extra>",
        visible=True if month == 'Jan' else False  # Show only January by default
    )
    fig.add_trace(scatter, row=1, col=1)
    scatter_traces.append(scatter)

# -----------------------------
# Heatmap: Seasonal Trends by Category
# -----------------------------
seasonal = data.groupby(['category', 'month'])['sales_volume'].mean().reset_index()
pivot = seasonal.pivot(index='category', columns='month', values='sales_volume')

heatmap = go.Heatmap(
    z=pivot.values,
    x=pivot.columns,
    y=pivot.index,
    colorscale='Blues',
    colorbar=dict(title="Avg Sales Volume"),
    hovertemplate="Category: %{y}<br>Month: %{x}<br>Avg Sales: %{z}<extra></extra>"
)
fig.add_trace(heatmap, row=1, col=2)

# -----------------------------
# Dropdown Menu for Month Filter
# -----------------------------
dropdown_buttons = []
for i, month in enumerate(months):
    # Control visibility of scatter plots
    visibility = [False] * len(months)
    visibility[i] = True  # Only selected month visible
    # Add heatmap visibility (always visible)
    visibility.append(True)

    dropdown_buttons.append(
        dict(
            label=month,
            method="update",
            args=[{"visible": visibility},
                  {"title": f"E-commerce Sales Dashboard - {month}"}]
        )
    )

# -----------------------------
# Layout and Display
# -----------------------------
fig.update_layout(
    updatemenus=[
        dict(
            buttons=dropdown_buttons,
            direction="down",
            showactive=True,
            x=0.35, y=1.15,
            xanchor="center", yanchor="top"
        )
    ],
    title="E-commerce Sales Dashboard - Jan",
    template="plotly_white",
    height=600
)

fig.update_xaxes(title_text="Sales Volume", row=1, col=1)
fig.update_yaxes(title_text="Revenue", row=1, col=1)
fig.update_xaxes(title_text="Month", row=1, col=2)
fig.update_yaxes(title_text="Product Category", row=1, col=2)

fig.show()
